In [1]:
import numpy as np

In [2]:
import jax.numpy as jnp

In [3]:
import time

In [4]:
from dataset_utils import load_dataset
from baselines_skylines import result_table, make_baselines_skylines
from crm_dataset import CRMDataset
from crm_model import Model

In [5]:
for dataset_name in ('scene', 'yeast', 'tmc2007',):
    
    print('*'*30, dataset_name, '*'*30)
    
    X_train, y_train, X_test, y_test, labels = load_dataset(dataset_name)
    
    print('learning baselines')
    pi0, pistar = make_baselines_skylines(dataset_name, X_train, y_train)
    
    sampling_probas = pi0.predict_proba(X_train)
    sampling_probas = np.array([_[:,1] for _ in sampling_probas]).T
    
    log_props_per_instance = jnp.log(sampling_probas).sum(axis=1)
    n_zero_props = (jnp.exp(log_props_per_instance) == 0).astype(int).sum()
    print(n_zero_props, '/', sampling_probas.shape[0], 'train instances have 0 propensity')
    assert (n_zero_props / sampling_probas.shape[0]) < .1
    
    poem_lambda = {'scene':.5, 'yeast':.5, 'tmc2007':.5}[dataset_name]
    
    ips_losses = []
    poem_losses = []
    
    start = time.time()
    for i in range(10):
        elapsed = int(time.time() - start)
        print('split', i, 'elapsed:', elapsed, 's', end=' ')

        # building CRM dataset
        np.random.seed(i*42) # controlling action randomness   
        crm_dataset = CRMDataset()
        crm_dataset.update_from_supervised_dataset(
            X_train, y_train, sampling_probas, 
            n_samples=4
        )        
        # print("Dataset ready:", len(crm_dataset), 'hash:', crm_dataset.actions.sum(), file=sys.stderr)

        # IPS
        ips_model = Model.null_model(X_test.shape[1], y_test.shape[1])
        ips_model.fit(
            crm_dataset, 
            tol=1e-6,
            maxiter=1000,
            verbose = 0, 
            lambda_ = 0,
            clip = 5e4,
            snips = 1,
        )
        # print("%2d) IPS     E[HL]: %.3f | CRM Loss (++data): %.3f | CRM Loss: %.3f" % (
        #     i,
        #     ips_model.expected_hamming_loss(X_test, y_test), 
        #     ips_model.crm_loss(ref_crm_dataset), 
        #     ips_model.crm_loss(crm_dataset)
        # ))
        ips_losses += [ips_model.expected_hamming_loss(X_test, y_test)]    

        # POEM   
        poem_model = Model.null_model(X_test.shape[1], y_test.shape[1])
        poem_model.fit(
            crm_dataset, 
            tol=1e-6,
            maxiter=1000,
            verbose=0, 
            clip = 5e4,
            lambda_ = poem_lambda, 
            snips = 1,
        )
        # print("%2d) POEM    E[HL]: %.3f | CRM Loss (++data): %.3f | CRM Loss: %.3f" % (
        #     i,
        #     poem_model.expected_hamming_loss(X_test, y_test), 
        #     poem_model.crm_loss(ref_crm_dataset), 
        #     poem_model.crm_loss(crm_dataset)
        # ))
        poem_losses += [poem_model.expected_hamming_loss(X_test, y_test)]    
    print()

    ips_losses = np.array(ips_losses)
    poem_losses = np.array(poem_losses)

    diff = ips_losses - poem_losses

    print('CI(L(IPS) - L(POEM)) @ alpha=5%%: [%.4f ; %.4f]' % (
        diff.mean() - 1.64*np.std(diff)/np.sqrt(10), diff.mean() + 1.64*np.std(diff)/np.sqrt(10)
    ))
    result_table(dataset_name, pi0, pistar, X_test, y_test, ips_loss=np.mean(ips_losses), poem_loss=np.mean(poem_losses))
    print('*'*80)
    print()

****************************** scene ******************************
X_train: (1805, 295) y_train: (1805, 6)
learning baselines


0 / 1805 train instances have 0 propensity
split 0 elapsed: 0 s split 1 elapsed: 1 s split 2 elapsed: 3 s split 3 elapsed: 4 s split 4 elapsed: 6 s split 5 elapsed: 7 s split 6 elapsed: 9 s split 7 elapsed: 10 s split 8 elapsed: 12 s split 9 elapsed: 14 s 
CI(L(IPS) - L(POEM)) @ alpha=5%: [-0.0315 ; -0.0049] 

********************************************************************************
Baseline -- crm paper: 0.257 -- vrcrm paper: 0.315 -- ours: 0.352
IPS      -- crm paper: 0.199 -- vrcrm paper: 0.225 -- ours: 0.205
POEM     -- crm paper: 0.195 -- vrcrm paper: 0.195 -- ours: 0.223
Skyline  -- crm paper: 0.110 -- vrcrm paper: 0.237 -- ours: 0.144
********************************************************************************

****************************** yeast ******************************
X_train: (1812, 104) y_train: (1812, 14)
learning baselines
0 / 1812 train instances have 0 propensity
split 0 elapsed: 0 s split 1 elapsed: 1 s split 2 elapsed: 2 s split 3 elapsed: 4 s split 